In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from IPython.display import Image
import regex as re
from IPython.core.display import display,HTML
from joblib import Parallel, delayed
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()

In [ ]:
url = 'https://www.acunr.es/perros/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

links= []

for e in tqdm(range(5)):
    muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
    uno = muchos.find_elements(By.TAG_NAME, 'a')
    links += (e.get_attribute('href') for e in uno)
    otra = driver.find_element(By.XPATH, '//*[@id="main"]/ul[1]/li[8]/a')
    otra.click()
'''
ESTO NO LO PONGO DENTRO DEL BUCLE PORQUE ES LA ULTIMA PAG
SI LO PONGO DENTRO DEL BUCLE, INTENTA DAR A CLICK Y ME DA ERROR
PORQUE NO HAY OTRA PAG
'''
muchos = driver.find_element(By.CLASS_NAME, 'col-lg-12')
uno = muchos.find_elements(By.TAG_NAME, 'a')
links += [e.get_attribute('href') for e in uno]


links_acunr= []
for e in range(len(links)):
    if e % 2:
        links_acunr.append(links[e])
    else:
        pass
    
    

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_acunr):
        protec_id.append(2) 
        driver.get(e)
        time.sleep(0.2)
        nombres.append(driver.find_element(By.TAG_NAME, 'h2').text)
        box = driver.find_element(By.CLASS_NAME, 'col-lg-12')
        img= box.find_element(By.TAG_NAME, 'img')
        imagen.append(img.get_attribute('src'))
        desc = box.find_elements(By.CLASS_NAME, 'card-text')
        for i in desc:
            if len(desc) == 12:
                desc.insert(9, i)
            else:
                pass
            
            i = i.text
            i = i.split(':', 1)
            i.pop(0)
            carac += i
            
            
            
driver.quit()

n = 13
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

In [ ]:
colnames= ('ID', 'EDAD', 'RAZA', 'SEXO', 'ORIGEN', 'CARACTER', 'APTO_CON_GATOS', 'LEHISMANIA', '4DX', 'SALUD', 'SITUACION', 'TAMAÑO', 'OBSERVACIONES')
acunr_ini = pd.DataFrame(caracter, columns=colnames)
acunr_ini['PROTECTORA']=protec_id
acunr_ini['NOMBRE']= nombres
acunr_ini['LINK']= links_acunr

images1 = imagen
acunr_ini['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


# display(HTML(acunr_ini.to_html(escape=False ,formatters=format_dict)))

acunr = acunr_ini.copy()

años= []
for e in acunr.EDAD:
    e = e.lower()
    if ' adulto' in e or ' senior' in e or ' yayo' in e:
        años.append(int(re.findall('[0-9]+', e)[0]))
    elif ' cachorro' in e:
        años.append(0)
    else:
        años.append(e)
acunr['AÑOS'] = años

raza= []
for e in acunr.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
acunr['RAZA']= raza

sexo= []
for e in acunr.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
acunr['SEXO']= sexo

tam= []
for e in acunr.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append((re.findall('[a-z,ñ]+', e)[0]))
acunr['TAMAÑO']= tam

nom= []
for e in acunr.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
acunr['NOMBRE']= nom


acunr.drop(['EDAD', 'ID', 'ORIGEN', 'APTO_CON_GATOS', 'LEHISMANIA', '4DX', 'SALUD', 'SITUACION', 'OBSERVACIONES'], axis= 1, inplace = True)